### Ingestion de la carpeta "movie_language"


In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Leer los archivos JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movies_languages_schema = StructType(fields=[
     StructField("movieId", IntegerType(), True),
     StructField("languageId", IntegerType(), True),
     StructField("languageRoleId", IntegerType(), True)
])

In [0]:
movies_languages_df = spark.read \
                   .schema(movies_languages_schema) \
                   .option("multiLine", True) \
                   .json(f"{bronze_folder_path}/{v_file_date}/movie_language") 

### Paso 2 - Renombrar las columnas y añadir columnas nuevas
1. "movieId" renombrar a "movie_id"
2. "languageId" renombrar a "language_id"
3. Agregar las columnas de "ingestion_date" y "envoronment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_languages_with_columns_df = add_ingestion_date(movies_languages_df) \
                                   .withColumn("environment", lit(v_environment)) \
                                   .withColumn("file_date", lit(v_file_date)) \
                                    .withColumnsRenamed(
                                        {"movieId": "movie_id", 
                                        "languageId": "language_id"
                                        })
                         

### 3 - Eliminar la columna "languageRoleId"

In [0]:
from pyspark.sql.functions import col

In [0]:
movies_languages_final_df = movies_languages_with_columns_df.drop(col("languageRoleId"))

### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_languages_final_df,"movie_silver","movies_languages","file_date")

In [0]:
# movies_languages_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_languages")

# Crear una tabla en base al data frame movies_languages_final_df
# Creara la tabla movies_languages en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver

#movies_languages_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_languages")

# Append, agrega los registros a la tabla existente, carga incremental de datos

#movies_languages_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_languages")

#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(movies_languages_final_df,"movie_silver","movies_languages",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
-- Consultar la DB movie_silver, tabla movies_languages
SELECT file_date, COUNT(1)
FROM movie_silver.movies_languages
GROUP BY 1;


In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_languages"))
display(spark.read.format("delta").load(f"{silver_folder_path}/movies_languages"))

In [0]:
dbutils.notebook.exit("Success")